In [ ]:
from Twitter_keeper import PullTweetsData

puller = PullTweetsData()

test_cases = [
            ("Hello! World🌍 https://www.example.com", "Hello/World"),
            ("Hello 😀 World 🚀🚀https://www.example.com", "Hello/World"),
            ("p https://www.example.com Hello World", "Hello/World"),
            ("#ทรงอย่างแบด ลูกสาวชอบมากถึงขนาดให้เก็บซองเลย์ที่เค้านั่งกินระหว่างรอพี่ๆpaper planesมา ไม่รู้ว่าอีกหน่อยเค้าจะลืมมั้ย แต่ก็ทำให้ตามที่เค้าขอ เพราะความสุขของแต่ละคนไม่เหมือนกัน ถ้าศิลปินคือแรงบันดาลใจให้ลูกเรากลายเป็นเด็กดีคนดี แม่อย่างเราก็ต้องขอบคุณเค้ามากๆนะ https://t.co/oWconpN1iP", "Hello/World"),
        ]
for text, expected_output in test_cases:
    print(puller.preprocessText(text))

In [ ]:
from Twitter_keeper import PullTweetsData
from unittest.mock import MagicMock
import pymongo
# create a fake MongoClient
fake_client = MagicMock(spec=pymongo.MongoClient)

# create a fake database
fake_db = MagicMock()
fake_collection = MagicMock()
# set the side effect for the fake MongoClient
fake_client.__getitem__.side_effect = lambda x: fake_db
fake_db.__getitem__.return_value = fake_collection
# use the fake MongoClient in your code
data_class = PullTweetsData()
data_class.connectToDB("test_database", "test_collection")
data_class._PullTweetsData__db = fake_client

# generate fake tweet data using fake-factory
fake_tweet = {
            "tweet_create_at": "2022-01-01",
            "tweet_author": "test1",
            "tweet_content": "This is from test1"
        }

# test the saveTweetsDict method
data_class.saveTweetsDict(fake_tweet)

# assert that the update_one method was called once
assert fake_db.update_one.call_count == 1

# assert that the update_one method was called with the expected arguments
assert fake_db.update_one.call_args[0] == ({"tweet_create_at": fake_tweet["tweet_create_at"], "tweet_author": fake_tweet["tweet_author"]},)
assert fake_db.update_one.call_args[1]["$set"] == fake_tweet
assert fake_db.update_one.call_args[1]["upsert"] is True

In [ ]:
pip install mongomock

In [ ]:
import unittest
from unittest.mock import MagicMock
import pymongo
from mongomock import MongoClient

from Twitter_keeper import PullTweetsData

Puller = PullTweetsData()
mock_cli = MongoClient()
mock_db = mock_cli['test_db']['test_col']
Puller._PullTweetsData__db = mock_db

tweet_post1 = {"tweet_create_at": "2022-01-01 12:00:00",
                "tweet_author": "test_author1",
                "tweet_text": "test tweet1"}

tweet_post2 = {"tweet_create_at": "2022-01-01 12:00:01",
                      "tweet_author": "test_author1",
                      "tweet_text": "test tweet2"}

Puller.saveTweetsDict(tweet_post=tweet_post1)
Puller.saveTweetsDict(tweet_post=tweet_post1)
Puller.saveTweetsDict(tweet_post=tweet_post2)

saved_tw = mock_db.find({
                "tweet_author": "test_author1"})
print((saved_tw))




In [ ]:
import time
import sched
from threading import Thread
from threading import Lock
import tweepy
import pandas as pd
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from sklearn.feature_extraction.text import CountVectorizer
import emoji
import re
import pymongo
from datetime import datetime, timezone
from dateutil import tz
import pytz
from termcolor import colored
from tqdm import tqdm
import os
from dotenv import load_dotenv
from urllib.parse import urlparse

In [ ]:
from pythainlp import sen

In [ ]:
pip install python-dotenv

In [41]:
from Twitter_keeper import PullTweetsData
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import re
import emoji
import numpy as np
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
import pandas as pd

class FindTopWord(PullTweetsData):

    def tokenize(self,d):  
        result = d.split("/")
        result = list(filter(None, result))
        return result

    def prepared_Text(self,text_list):
        new_text = []
        for text in text_list:
            new_text.append(self.preprocessText(text))
        return new_text
        
    def MostWordFinder(self,tweets_list):
        vectorizer = CountVectorizer(tokenizer=self.tokenize)
        transformed_data = vectorizer.fit_transform(tweets_list)
        keyword_df1 = pd.DataFrame(columns = ['word', 'count'])
        keyword_df1['word'] = vectorizer.get_feature_names_out()
        print(vectorizer.get_feature_names_out())
        keyword_df1['count'] = np.ravel(transformed_data.sum(axis=0))   
        keyword_df1.sort_values(by=['count'], ascending=False).head(10)
        return keyword_df1



class SentimentAnalyze(PullTweetsData):

    def __init__(self):
        self.__df_train = pd.read_csv("general-amy.csv")
        self.__vectorizer = CountVectorizer()
        self.__model = MultinomialNB()

    def preprocess_train_text(self,text):
        text = self.removeLink(text)
        text = self.removeEmoji(text)
        text = self.removeSpecialChar(text)
        final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
        final = word_tokenize(final, engine="newmm")
        final = " ".join(word for word in final)
        # final = " ".join(word for word in final.split() if word.lower not in thai_stopwords())
        return final
        # tokens = word_tokenize(text, engine="newmm")
        # result = [word for word in tokens if word not in list(
        #         thai_stopwords()) and " " not in word]
        # return " ".join(result).rstrip()

    def run_prep_train(self):
        self.__df_train['text'] = self.__df_train['text'].apply(self.preprocess_train_text)

    def split_training(self):
        self.X = self.__vectorizer.fit_transform(self.__df_train['text'])
        self.y = self.__df_train['sentiment']
        # Split the data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2)

    def training_model(self):
        self.run_prep_train()
        self.split_training()
        # Train the model
        self.__model.fit(self.X_train, self.y_train)

    def evaluating_model(self):
        #Evaluate the model on the test data
        y_pred = self.__model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        return accuracy

    def sentiment_analyzer(self,text):
        #Use the model to make predictions on new data
        new_text = self.__vectorizer.transform([text])
        new_bag_of_word = self.__vectorizer.transform(pd.Series([self.preprocess_train_text(text)]))
        # print(new_bag_of_word)
        new_pred = self.__model.predict(new_bag_of_word)
        return new_pred[0]

class main(PullTweetsData):
    def __init__(self):
        self.find_top_word = FindTopWord()
        self.sentiment_analyze = SentimentAnalyze()
        self.pull_tweets = PullTweetsData()

    def load_sample_tweets(self,author="",keyword="",hashtag="",location="",text="",fromTime="",toTime=""):
        self.pull_tweets.connectToDB("twitter_keeper","tweets")
        return self.pull_tweets.find_multi(author,keyword,hashtag,location,text,fromTime,toTime)

    def tweets_find_top_word(self,author="",keyword="",hashtag="",location="",text="",fromTime="",toTime=""):
        tweets_list = self.prepared_Text(self.load_sample_tweets(author,keyword,hashtag,location,text,fromTime,toTime))
        return self.find_top_word.MostWordFinder(tweets_list)

    def tweets_sentiment_analyzer(self,author="",keyword="",hashtag="",location="",text="",fromTime="",toTime=""):
        self.sentiment_analyze.training_model()
        acc = self.sentiment_analyze.evaluating_model()
        tweets_list = self.load_sample_tweets(author,keyword,hashtag,location,text,fromTime,toTime)
        df = pd.DataFrame({'text':[],'sentiment':[]})
        for tweet in tqdm(tweets_list):
            sentiment = self.sentiment_analyze.sentiment_analyzer(tweet['text'])
            df = pd.concat([df,pd.DataFrame(pd.Series([tweet['text'],sentiment], index=df.columns)).T],ignore_index=True)
        return df

TweetsCounter = main()

In [33]:
TweetsCounter.load_sample_tweets(keyword="#Onet66",fromTime="2023.2.13.3.53.53",toTime="2024.2.13.3.53.53")

[]

In [34]:
TweetsCounter.splittime("2023.2.13.3.53.53")

[2023, 2, 13, 3, 53, 53]

In [38]:
TweetsCounter.find_multi("","","","Bangkok","","2023.2.13.3.24.45","2024.2.13.3.24.45")

[]

In [42]:
from IPython.display import display,HTML

df = TweetsCounter.tweets_sentiment_analyzer(keyword="#Onet66",fromTime="2023.2.12.00.00.00",toTime="2024.2.13.11.00.00")
df

100%|██████████| 6316/6316 [00:03<00:00, 1865.79it/s]


,text,sentiment
0,🔮คาถาขอให้สอบผ่าน🔮\nตั้ง นะโม ตัสสะ ภะคะวะโต อ...,neg
1,🔮คาถาขอให้สอบผ่าน🔮\nตั้ง นะโม ตัสสะ ภะคะวะโต อ...,neg
2,💡ราศีที่มีดวงจะสมหวังในความรัก💡\n✅พฤษภ\n✅มังกร...,neg
3,รีวิวค่ะ #ดูไพ่กับoctsy\n\nรีวิว ดูดวงไพ่ 🙏\n\...,pos
4,คิวว่างทัก DM เลย \nคำถามละ32฿\n🥹ราคานี้เฉพา...,neg
...,...,...
6311,ถ้าได้ อังกฤษ วิท คฺณิต ภาษาไทย \n 50+/100 แ...,neg
6312,วิทย์กับอังกฤษถ้าได้50+ แจก50/1 คน รียาวๆ55 #O...,neg
6313,ทุกวิชา50++ แจก 50บ./1 อังกฤษ/ไทย 80++ แจกอีก ...,pos
6314,อังกฤษได้50+++ แจก100 #Onet66,pos


In [8]:
import pandas as pd
import plotly.express as px

# Count the number of occurrences of each sentiment
sentiment_counts = df['sentiment'].value_counts()

print(sentiment_counts)
# Calculate the percentage of each sentiment
sentiment_percentages = sentiment_counts / len(df) * 100
print(sentiment_percentages)
# Create the pie chart
fig = px.pie(sentiment_percentages, values=sentiment_percentages.values, 
             names=sentiment_percentages.index, title='Sentiment Percentage')

# Show the chart
fig.show()

neg    4001
pos    2315
Name: sentiment, dtype: int64
neg    63.347055
pos    36.652945
Name: sentiment, dtype: float64


In [6]:
pip install plotly-express

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 3.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 39.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
print(TweetsCounter.tweets_sentiment_analyzer("#ใส่นัวแฟมิลี่"))

In [ ]:
import pandas as pd
df = pd.read_csv("general-amy.csv")
print(df['text'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import re
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

def removeSpecialChar(text):
    return re.sub(r"[\]\[!-@#$?%+:\"\n^_‘’“”]", "", text).rstrip()

def removeEmoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.EMOJI_DATA]
    return ''.join([str for str in allchars if not any(i in str for i in emoji_list)]).rstrip()

def removeLink(text):
    new_text = re.sub(r'https?:\/\/[^\s]+', '', text)
    # text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE).rstrip()
    link_regex = r"(https?:\/\/[-a-zA-Z0-9@:%._\+~#=]+)"
    return re.sub(link_regex, '', new_text).rstrip().lstrip()

# Pre-processing the data
def preprocess_train_text(text):
    text = removeLink(text)
    text = removeEmoji(text)
    text = removeSpecialChar(text)
    tokens = word_tokenize(text, engine="newmm")
    result = [word for word in tokens if word not in list(
            thai_stopwords()) and " " not in word]
    return " ".join(result).rstrip()

df["text"] = df["text"].apply(preprocess_train_text)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

#Evaluate the model on the test data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)





In [ ]:
#Evaluate the model on the test data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#Use the model to make predictions on new data
new_text = ['''เราชอบคอมเม้นนี้จัง 🥹 Cr ภาพจากเพจ IHaveCPU #hrk #heartrocker #ใส่นัวแฟมิลี่ https://t.co/7QHIs4fYsd''']
new_text = vectorizer.transform(new_text)
new_pred = model.predict(new_text)
print("Sentiment:", new_pred[0])

In [ ]:
from tqdm import tqdm
import os
from datetime import datetime, timezone
from dotenv import load_dotenv
from dateutil import tz
import tweepy

API_KEY = "SnxucIPt1fg7UUyVOT0T5j0pR"
API_KEY_SECRET = "yaToQPv95OA1fiNTHD8drKM8g8rZGM7jSQnPOLoxU3QA9UpaLm"
ACCESS_TOKEN = "1722424471-Xb0DjPVOqXsLj2sXEYXmU2sqxaDC4B793erGO6J"
ACCESS_TOKEN_SECRET = "D2LyXN11zAoZB0M476eb1ZGDM55oRvy4tBWNb8pR4CO0h"

auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)
count = 0
def getHashtag(entity_hashtag):
        hashtag = ""
        for i in range(0, len(entity_hashtag)):
            hashtag = hashtag + "#"+entity_hashtag[i]["text"]
        return hashtag
def createDictData(tweet_author, tweet_create_at, hashtag, keyword, text):
        tweet = {}
        tweet["tweet_author"] = tweet_author
        tweet["tweet_create_at"] = tweet_create_at
        tweet["hashtag"] = hashtag
        tweet["keyword"] = keyword
        tweet["text"] = text
        return tweet
amount = 10
query = "#Valorant"
for tweet in tqdm(tweepy.Cursor(api.search_tweets, q=query, count=100,
                                        result_type="recent", tweet_mode='extended').items()):
    entity_hashtag = tweet.entities.get('hashtags')
    hashtag = getHashtag(entity_hashtag)
    tweet_author = tweet.user.screen_name
    keyword = query
    dt_str = str(tweet.created_at)
    format = "%Y-%m-%d %H:%M:%S%z"
    dt_utc = datetime.strptime(dt_str, format)
    local_zone = tz.tzlocal()
    dt_local = dt_utc.astimezone(local_zone)
    tweet_create_at = dt_local
    try:
        text = tweet.retweeted_status.full_text
    except:
        text = tweet.full_text
    user = tweet.user
    location = user.location
    print(user.screen_name,location)
    count += 1
    if count == amount:
        count = 0
        break
